<a href="https://colab.research.google.com/github/SergioMendonca1/IMERSAO_IA_2025/blob/main/Sr_Doutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. INSTALAÇÃO DAS BIBLIOTECAS
print("Instalando bibliotecas (google-generativeai e google-adk)...")
%pip install -q google-generativeai
%pip install -q google-adk
print("Bibliotecas instaladas.")


Instalando bibliotecas (google-generativeai e google-adk)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/

In [2]:
# 2. IMPORTS E CONFIGURAÇÃO DA API KEY
print("\nConfigurando API e importando módulos...")
import os
from google.colab import userdata
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types as genai_types
from datetime import date
import textwrap
from IPython.display import display, Markdown # Certifique-se que está importado
import warnings
import time # Mantendo para o time.sleep()

warnings.filterwarnings("ignore")

# Configura a API Key do Google
GOOGLE_API_KEY_FROM_USERDATA = None
try:
    GOOGLE_API_KEY_FROM_USERDATA = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    print(f"Aviso: Não foi possível obter GOOGLE_API_KEY de userdata: {e}")

if GOOGLE_API_KEY_FROM_USERDATA:
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY_FROM_USERDATA
    print("API Key carregada de userdata e DEFINIDA COMO VARIÁVEL DE AMBIENTE ('GOOGLE_API_KEY').")
    try:
        genai.configure(api_key=GOOGLE_API_KEY_FROM_USERDATA)
        print("Configuração global genai.configure() chamada com sucesso.")
    except Exception as e:
        print(f"Aviso: Erro ao chamar genai.configure(): {e}")
else:
    print("ERRO CRÍTICO: GOOGLE_API_KEY não encontrada nas secrets do Colab (userdata).")
    raise ValueError("API Key não configurada. Verifique as secrets do Colab.")

if 'GOOGLE_API_KEY' in os.environ and os.environ['GOOGLE_API_KEY']:
    print("Confirmação: Variável de ambiente GOOGLE_API_KEY está presente e não vazia.")
else:
    print("ALERTA: Variável de ambiente GOOGLE_API_KEY NÃO está presente ou está vazia após a tentativa de configuração!")

MODEL_ID = "gemini-1.5-flash-latest"
print(f"Modelo a ser usado pelo agente: {MODEL_ID}")



Configurando API e importando módulos...
API Key carregada de userdata e DEFINIDA COMO VARIÁVEL DE AMBIENTE ('GOOGLE_API_KEY').
Configuração global genai.configure() chamada com sucesso.
Confirmação: Variável de ambiente GOOGLE_API_KEY está presente e não vazia.
Modelo a ser usado pelo agente: gemini-1.5-flash-latest


In [6]:
# 3. FUNÇÕES AUXILIARES
def to_markdown(text): # Função original para Markdown
    lines = [line.strip() for line in text.split('\n')]
    processed_lines = []
    for line in lines:
        if line:
            if line.startswith('• '): line = '* ' + line[2:]
            elif line.startswith('- '): line = '* ' + line[2:]
            if not line.startswith('> ') and not line.startswith('* ') and not line.startswith('  * '):
                 processed_lines.append('> ' + line)
            else:
                 processed_lines.append(line)
        else:
            processed_lines.append(line)
    text_final = "\n".join(processed_lines).strip()
    return Markdown(text_final)

print("Funções auxiliares definidas.")


Funções auxiliares definidas.


In [3]:

# 4. DEFINIÇÃO DO AGENTE DE MEDICAMENTOS
print("\nDefinindo o agente de medicamentos...")
agente_medicamentos = None
instrucoes_agente_ajustadas = (
    "Você é um assistente farmacêutico virtual interativo, preciso e prestativo. "
    "Siga este fluxo CUIDADOSAMENTE:\n"
    "1. O usuário fornecerá uma lista de nomes de medicamentos.\n"
    "2. Sua PRIMEIRA resposta DEVE SER apenas para confirmar os medicamentos que você identificou. Liste-os. Ex: 'Entendi que você mencionou: Novalgina, AAS. Correto? (sim/não)'.\n"
    "3. AGUARDE a confirmação ('sim' ou 'não').\n"
    "4. Se 'sim': Na sua PRÓXIMA resposta, PERGUNTE se ele deseja ver os detalhes. Ex: 'Ótimo! Gostaria de ver os detalhes da(s) bula(s) para [lista de medicamentos]? (sim/não)'.\n"
    "5. Se 'não' na etapa 3: Peça para ele fornecer a lista correta e reinicie da etapa 2.\n"
    "6. AGUARDE a resposta para os detalhes (etapa 4).\n"
    "7. Se 'sim' para ver detalhes: Na sua PRÓXIMA resposta, para CADA medicamento confirmado, forneça as seguintes informações, separando claramente cada item com um título em negrito:\n"
    "   - **Nome do Medicamento**\n"
    "   - **Principal Indicação:** (Para que serve primariamente)\n"
    "   - **Contraindicações Principais:** (Quem NÃO DEVE usar ou em que situações. Se não houver contraindicações absolutas muito fortes, mencione precauções importantes como se fossem contraindicações relativas.)\n"
    "   - **Efeitos Colaterais Comuns:** (Liste 2-4 dos mais frequentes e menos graves)\n"
    "   - **Advertências Importantes/Efeitos Graves (se houver):** (Qualquer aviso crucial, como interações, reações adversas sérias, ou precauções críticas)\n"
    "   - **Nome Genérico (se aplicável e conhecido):**\n"
    "   Use uma linha em branco entre cada medicamento detalhado. Após detalhar TODOS os medicamentos, vá para a etapa 8.\n"
    "8. Se 'não' para ver detalhes (na etapa 6), OU APÓS fornecer todos os detalhes (da etapa 7): Na sua PRÓXIMA resposta, PERGUNTE ao usuário: 'Entendido. Para que eu possa te ajudar a ter uma ideia de onde comprar e uma estimativa de preço, por favor, me informe seu CEP no formato XXXXX-XXX ou sua cidade/bairro.'\n"
    "9. AGUARDE o usuário informar o CEP ou cidade/bairro.\n"
    "10. Após o usuário informar a localização: Na sua PRÓXIMA resposta, faça o seguinte para CADA medicamento:\n"
    "    a. Diga: 'Para encontrar [Nome do Medicamento] próximo a [Localização informada], sugiro que você:\n"
    "       - Utilize aplicativos de mapas (como Google Maps, Waze) e pesquise por \"farmácias\" ou \"drogarias\" para ver as opções mais próximas da sua localização e seus endereços.\n"
    "       - Verifique os sites de grandes redes de farmácias (ex: Drogasil, Droga Raia, Pague Menos, Drogaria São Paulo), pois muitas oferecem busca por CEP para encontrar lojas ou delivery.'\n"
    "    b. Sobre o preço, diga: 'Quanto ao preço de [Nome do Medicamento], ele pode variar bastante dependendo da farmácia, marca (se genérico ou de referência) e promoções. Uma busca online por \"preço [Nome do Medicamento]\" ou a consulta direta nos sites e aplicativos das farmácias pode te dar uma estimativa mais precisa. [Mencione se é um medicamento geralmente acessível ou se tende a ser mais caro, se tiver essa noção geral, MAS SEMPRE COM RESSALVA DE VARIAÇÃO]. Não se esqueça de pesquisar e comparar preços!'\n"
    "    c. Adicione a ressalva: 'Lembre-se que estas são apenas sugestões. A disponibilidade e os preços exatos devem ser verificados diretamente com as farmácias.'\n"
    "11. Após fornecer essas informações para TODOS os medicamentos, pergunte: 'Posso ajudar com mais alguma coisa? (sim/não)'.\n"
    "12. Se o usuário responder 'não' à pergunta da etapa 11, responda: 'Ok. Até mais! (Para finalizar completamente, digite sair)'. Se responder 'sim', pergunte 'Como posso ajudar?'.\n"
    "13. Siga EXATAMENTE estas etapas. AGUARDE a entrada do usuário entre cada resposta sua. Responda em português brasileiro."
)

try:
    agente_medicamentos = Agent(
        name="AnalisadorDeMedicamentosAjustado",
        instruction=instrucoes_agente_ajustadas,
        model=MODEL_ID,
    )
    print("Sr. Doutor 'AnalisadorDeMedicamentosAjustado' definido com sucesso!")
except Exception as e:
    print(f"ERRO ao definir o agente: {e}")



Definindo o agente de medicamentos...
Sr. Doutor 'AnalisadorDeMedicamentosAjustado' definido com sucesso!


In [7]:
# 5. INTERAÇÃO COM O USUÁRIO
if agente_medicamentos:
    print("\n--- Consulta Interativa de Medicamentos ---")
    print("Você pode digitar 'sair' a qualquer momento para terminar a conversa.")

    session_service = InMemorySessionService()
    user_id = "user_final_meds"
    current_session_id = f"session_final_meds_{date.today().strftime('%Y%m%d')}_{os.urandom(4).hex()}"
    session = session_service.create_session(app_name=agente_medicamentos.name, user_id=user_id, session_id=current_session_id)
    runner = Runner(agent=agente_medicamentos, app_name=agente_medicamentos.name, session_service=session_service)
    print(f"Sessão iniciada: {current_session_id}")

    primeira_mensagem_usuario = True
    while True:
        if primeira_mensagem_usuario:
            print("\nOlá! Sou o Sr. Doutor, seu assistente de medicamentos. Quais medicamentos você gostaria de consultar hoje? (Ex: Novalgina, AAS)")
            user_input = input("Você: ").strip()
            if not user_input:
                print("Sr. Doutor: Por favor, informe os nomes dos medicamentos para começar.")
                continue
            primeira_mensagem_usuario = False
        else:

            time.sleep(0.2)
            user_input = input("Você: ").strip()

        if user_input.lower() == 'sair':
            print("Sr. Doutor: Encerrando a conversa. Até logo!")
            break
        if not user_input:
            print("Sr. Doutor: Por favor, digite sua resposta ou 'sair' para terminar.")
            continue

        content = genai_types.Content(role="user", parts=[genai_types.Part(text=user_input)])

        print("\nSr. Doutor pensando...")
        agent_response_text = ""

        try:

            for event in runner.run(user_id=user_id, session_id=current_session_id, new_message=content):

                if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            agent_response_text += part.text
                            if not agent_response_text.endswith("\n"):
                                agent_response_text += "\n"

            if agent_response_text:
                print("\nSr. Doutor:")
                display(to_markdown(agent_response_text.strip())) # Voltando a usar display(to_markdown(...))
            else:
                print(f"\nSr. Doutor: Não recebi uma resposta textual processável desta vez.")

        except Exception as e:
            print(f"\nOcorreu um erro CRÍTICO durante a execução do agente: {e}")
            break
else:
    print("\nNão foi possível iniciar a consulta de medicamentos porque o agente não foi definido corretamente.")





--- Consulta Interativa de Medicamentos ---
Você pode digitar 'sair' a qualquer momento para terminar a conversa.
Sessão iniciada: session_final_meds_20250517_209aa84d

Olá! Sou o Sr. Doutor, seu assistente de medicamentos. Quais medicamentos você gostaria de consultar hoje? (Ex: Novalgina, AAS)
Você: novalgina

Sr. Doutor pensando...

Sr. Doutor:


> Entendi que você mencionou: Novalgina. Correto? (sim/não)

Você: sim

Sr. Doutor pensando...

Sr. Doutor:


> Ótimo! Gostaria de ver os detalhes da(s) bula(s) para Novalgina? (sim/não)

Você: sim

Sr. Doutor pensando...

Sr. Doutor:


...

Você: 04616-005

Sr. Doutor pensando...

Sr. Doutor:


> Para encontrar Novalgina próximo a 04616-005, sugiro que você:
* Utilize aplicativos de mapas (como Google Maps, Waze) e pesquise por "farmácias" ou "drogarias" para ver as opções mais próximas da sua localização e seus endereços.
* Verifique os sites de grandes redes de farmácias (ex: Drogasil, Droga Raia, Pague Menos, Drogaria São Paulo), pois muitas oferecem busca por CEP para encontrar lojas ou delivery.

> Quanto ao preço de Novalgina, ele pode variar bastante dependendo da farmácia, marca (se genérico ou de referência) e promoções. Uma busca online por "preço Novalgina" ou a consulta direta nos sites e aplicativos das farmácias pode te dar uma estimativa mais precisa.  A Novalgina é um medicamento geralmente acessível, mas o preço pode variar. Não se esqueça de pesquisar e comparar preços!

> Lembre-se que estas são apenas sugestões. A disponibilidade e os preços exatos devem ser verificados diretamente com as farmácias.

> Posso ajudar com mais alguma coisa? (sim/não)

Você: não

Sr. Doutor pensando...

Sr. Doutor:


> Ok. Até mais! (Para finalizar completamente, digite sair)

Você: sair
Sr. Doutor: Encerrando a conversa. Até logo!
